# Brainstat analysis comparing 3T and 7T epilepsy  

Surface-based comparisons  
- vertex-wise T-test : are controls and pts different
    - Use brainstat
- vertex-wise effect size : how big are the distances between the vertex differences?
    - Use own function


For figures: 
- Visualize effect size on a brain masked for significant p-values 

In [3]:
import pandas as pd
import numpy as np
import nibabel as nib
import brainstat as bstat

In [19]:
# specify root directories
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_mp": "micapipe_v0.2.0",
    "dir_hu": "hippunfold_v1.3.0/hippunfold",
    "study": "3T",
    "ID_ctrl" : ["HC"],
    "ID_Pt" : ["PX"]
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "study": "7T",
    "ID_col" : ["PNC", "Pilot"], # column for ID in demographics file
    }

studies = [MICs, PNI]

demographics = {
    "pth" : "/host/verges/tank/data/daniel/3T7T/z/data/pt/demo_22May2025.csv",
    # column names:
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "grp" : "grp_detailed" # col name for participant grouping variable of interest
}

px_grps = { # specify patient group labels to compare to controls
    'all' : ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'],
    'TLE' : ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'],
    'TLE_L': ['TLE_L', 'mTLE_L'],
    'TLE_R': ['TLE_R', 'mTLE_R'],
    'FCD' : ['FLE_R', 'FLE_L'],
    'MFCL' : ['MFCL', 'MFCL_bTLE'],
    'UKN' : ['UKN_L', 'UKN_U']
}

ctrl_grp = {'ctrl' : ['CTRL']}

surfaces = ["fsLR-5k"]
labels = ["pial", "white", "midThick"]

In [5]:
# retrieve surfaces from pt of interest
def load_surf(study, IDss):
    """
    Get the surface data for a given group.

    inputs:
    study: dictionary item with keys 'name', 'dir_root', 'study'
    IDs: pd.dataframe woth cols IDs and SES indicating all participants IDs to extract surfaces for

    outputs:
    surfs: pd.dataframe with vertices in rows and unique ID_SES in columns
    """
    
    import nibabel as nib
    
    # get the list of patients in the group
    pt_list = bstat.get_subjects(study, pt_grp)
    
    # get the surface data for each patient
    surf_data = {}
    for pt in pt_list:
        surf_data[pt] = {}
        for surf in surfaces:
            surf_data[pt][surf] = bstat.get_surface_data(study, pt, surf)
    
    return surf_data

In [6]:
# when working, add to Utils scripts
def get_surfPath(root, deriv_fldr, sub, ses, label="midthickness", surf="fsLR-5k", space="nativepro", hemi="LR"):
    """
    Get the path to the surface data for a given subject and session.
    Assumes BIDS format of data storage.

    inputs:
        root: root directory of the study
        deriv_fldr: name of derivative folder containing the surface data
        sub: subject ID (no `sub-` prefix)
        ses: session ID (with leading zero if applicable; no `ses-` prefix)
        surf: surface type and resolution (e.g., fsLR-32k, fsLR-5k)
        label: surface label (e.g., "pial", "white", "midThick")
        space: space of the surface data (e.g., "nativepro", "fsnative")
        hemi: hemisphere to extract (default is "LR" for both left and right hemispheres)

    outputs:
        path to the surface data files
    """

    # make surf to lower case
    label = label.lower()

    # ensure that surface is well defined
    if label == "pial":
        label = "pial"
    elif label == "white":
        label = "white"
    elif label == "midThick" or label == "midthickness":
        label = "midthickness"
    else:
        raise ValueError("Invalid surface type. Choose from 'pial', 'white', or 'midThick'.")
    
    # construct the path to the surface data file
    
    pth = f"{root}/derivatives/{deriv_fldr}/sub-{sub}/ses-{ses}/surf"
    
    hemi = hemi.upper()
    if hemi == "LEFT" or hemi == "L":
        hemi = "L"
    elif hemi == "RIGHT" or hemi == "R":
        hemi = "R"
    elif hemi != "LR":
        raise ValueError("Invalid hemisphere. Choose from 'L', 'R', or 'LR'.")

    # handle hippunfold naming convention
    if "hippunfold" in deriv_fldr.lower():
        # space usually: "T1w"
        # surf usually: "2mm"
        # label options: "hipp_outer", "hipp_inner", "hipp_midthickness"

        if hemi == "LR":
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            
            print(f"[surf_pth] Returning hippunfold paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_den-{surf}_label-{label}.surf.gii"

    elif "micapipe" in deriv_fldr.lower():
        if hemi == "LR":
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_surf-{surf}_label-{label}.surf.gii"
        
            pth = [pth_L, pth_R]
            print(f"[surf_pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            print(f"[surf_pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_surf-{surf}_label-{label}.surf.gii"

    return pth


In [18]:
demo = pd.read_csv(demographics['pth'], dtype=str)
demo

,MICS_ID,PNI_ID,study,SES,Date,scanDate,handedness,education,ethnicity,gender,...,Seizure onset (yr),"Genetic test (year,results)",Employment,EMU admission date(dd-mm-yy),FDG.PET,Gender,"Baseline MRI (year,results)",age,grp,grp_detailed
0,HC129,Pilot013,7T,05,18.04.2024,18.04.2024,L,Master Studnet,Canadian,NaN,...,NaN,NaN,MSc student,NaN,NaN,F,NaN,27.72895277207392,CTRL,CTRL
1,HC082,PNC003,7T,01,06.05.2022,06.05.2022,R,PhD Student,Japanese,NaN,...,NaN,NaN,PhD student,NaN,NaN,F,NaN,24.632443531827516,CTRL,CTRL
2,HC082,PNC003,7T,02,13.06.2022,13.06.2022,R,PhD Student,Japanese,NaN,...,NaN,NaN,PhD student,NaN,NaN,F,NaN,24.736481861738536,CTRL,CTRL
3,HC082,PNC003,7T,03,13.03.2023,13.03.2023,R,PhD Student,Japanese,NaN,...,NaN,NaN,PhD student,NaN,NaN,F,NaN,25.4839151266256,CTRL,CTRL
4,HC082,PNC003,7T,04,24.10.2023,24.10.2023,R,PhD Student,Japanese,NaN,...,NaN,NaN,PhD student,NaN,NaN,F,NaN,26.099931553730322,CTRL,CTRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,HC152,PNC039,3T,01,29.01.2025,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,PhD Student,NaN,NaN,M,NaN,NaN,CTRL,CTRL
120,PX215,PNE020,3T,01,20.03.2025,NaN,NaN,NaN,NaN,NaN,...,21,pending,Social worker (before),05.02.2025,2025:Normal study. No evidence of abnormal hyp...,F,2021:No significant abnormality is noted. In p...,58.12457221081451,UKN,UKN_U
121,PX216,PNE021,3T,01,26.03.2025,NaN,NaN,NaN,NaN,NaN,...,32,NaN,"arret de travaiile presentement, normalement s...",16.01.2025,2025:indeterminate mild hypometabolism in the ...,F,"2024: nonlesional, re-inspection D.B:right mes...",49.72210814510609,CTRL,CTRL
122,HC152,PNC039,3T,02,08.04.2025,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,PhD Student,NaN,NaN,M,NaN,NaN,CTRL,CTRL


In [24]:
label = "midthickness"
hemi = "L"
space = "nativepro"
surf = "fsLR-5k"

if 'surf_dfs' not in locals():
    surf_dfs = {}

for study in studies:
    if study['study'] == "3T":
        ID_col = demographics['ID_3T']
    elif study['study'] == "7T":
        ID_col = demographics['ID_7T']

    # Controls
    ctrl_labels = ctrl_grp['ctrl']
    print(ctrl_labels)
    print(demographics['grp'])
    # check column demographics['grp'] for values matching ctrl_labels
    ids = demo.loc[demo[demographics['grp']].isin(ctrl_labels), [ID_col, demographics['SES']]].copy()

    print(ids_ctrl)
    continue

    for i, row in ids_ctrl.iterrows():
        ID = row[ID_col]
        SES = row[demographics['SES']]
        print(f"[surf_pth] sub-{ID}_ses-{SES}")
        pth = get_surfPath(root=study['dir_root'], deriv_fldr=study['dir_mp'], sub=ID, ses=SES, label=label, surf=surf, space=space, hemi=hemi)
        # add this pth to the dataframe
        if isinstance(pth, list):
            ids_ctrl.loc[i, f'surf_L'] = pth[0]
            ids_ctrl.loc[i, f'surf_R'] = pth[1]
        else:
            ids_ctrl.loc[i, f'surf_{hemi}'] = pth 
    # add the control dataframe to the dictionary
    surf_dfs[(study['name'], 'ctrl')] = ids_ctrl

    # Patients (all px_grps)
    for grp_name, grp_labels in px_grps.items():
        ids_px = demo.loc[demo[demographics['grp']].isin(grp_labels), [ID_col, demographics['SES']]].copy()
        for i, row in ids_px.iterrows():
            ID = row[ID_col]
            SES = row[demographics['SES']]
            print(f"[surf_pth] sub-{ID}_ses-{SES}")
            pth = get_surfPath(root=study['dir_root'], deriv_fldr=study['dir_mp'], sub=ID, ses=SES, label=label, surf=surf, space=space, hemi=hemi)
            ids_px.loc[i, f'surf_{hemi}'] = pth
        
        if isinstance(pth, list):
            ids_ctrl.loc[i, f'surf_L'] = pth[0]
            ids_ctrl.loc[i, f'surf_R'] = pth[1]
        else:
            ids_ctrl.loc[i, f'surf_{hemi}'] = pth 

        surf_dfs[(study['name'], grp_name)] = ids_px


['CTRL']
grp_detailed
    MICS_ID    PNI_ID study SES        Date     scanDate handedness  \
0     HC129  Pilot013    7T  05  18.04.2024   18.04.2024          L   
1     HC082    PNC003    7T  01  06.05.2022  06.05.2022           R   
2     HC082    PNC003    7T  02  13.06.2022   13.06.2022          R   
3     HC082    PNC003    7T  03  13.03.2023   13.03.2023          R   
4     HC082    PNC003    7T  04  24.10.2023   24.10.2023          R   
..      ...       ...   ...  ..         ...          ...        ...   
118   HC083    PNC011    3T  02  28.01.2025   11.11.2022          R   
119   HC152    PNC039    3T  01  29.01.2025          NaN        NaN   
121   PX216    PNE021    3T  01  26.03.2025          NaN        NaN   
122   HC152    PNC039    3T  02  08.04.2025          NaN        NaN   
123   HC076    PNC018    3T  04  17.04.2025   28.09.2023          L   

           education  ethnicity gender  ... Seizure onset (yr)  \
0    Master Studnet    Canadian    NaN  ...                

In [15]:
surf_dfs

{('MICs',
  "{'ctrl': ['ctrl']}"): Empty DataFrame
 Columns: [MICS_ID, SES]
 Index: [],
 ('MICs',
  'ctrl'): Empty DataFrame
 Columns: [MICS_ID, SES]
 Index: []}

In [ ]:
# flip TLEs --> put all lesions on same side

In [ ]:
# get difference maps at 3T (3T ctrl - 3T cases)

In [ ]:
# get difference maps at 7T (7T ctrl - 7T cases)

In [ ]:
# get difference maps of difference maps (3T dif maps - 7T dif maps)